# Label to Distribution 

- batch label to gaussian pdf 

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.modules.loss import _WeightedLoss

In [11]:
x = torch.tensor([ [-2.7,-2,-1, 0, 1, 2, 2.7],[-1.7,-1,-0.5, 0, 0.5, 1, 1.7]])
print(x.shape)

def pdf_fn(x):
  x_pdf = torch.exp( -(x)**2 /2  ) * 1/( torch.pi * torch.sqrt(torch.tensor(2)) )
  return x_pdf

x_pdf = torch.exp( -(x)**2 /2  ) * 1/( torch.pi * torch.sqrt(torch.tensor(2)) )
print(x_pdf)

torch.Size([2, 7])
tensor([[0.0059, 0.0305, 0.1365, 0.2251, 0.1365, 0.0305, 0.0059],
        [0.0531, 0.1365, 0.1986, 0.2251, 0.1986, 0.1365, 0.0531]])


In [12]:
x_pdf

tensor([[0.0059, 0.0305, 0.1365, 0.2251, 0.1365, 0.0305, 0.0059],
        [0.0531, 0.1365, 0.1986, 0.2251, 0.1986, 0.1365, 0.0531]])

In [13]:
label = torch.tensor([[0,0,0, 0, 0, 1, 0],[0,0,0, 0, 0, 1, 0]])
print(label.shape)

torch.Size([2, 7])


In [59]:
def label_to_dist(label):
  gap_ = 1
  target_label_index = torch.where(label==1)[0][0].numpy()
  label_dist = [i for i in np.arange(target_label_index,0,-gap_)] + [i for i in np.arange(0,len(label) - target_label_index,gap_ )]
  return label_dist
#--
def label_to_dist_v1(label):
  gap_ = 1
  target_label_index = torch.where(label==1)[0][0].numpy()
  label_dist = [i for i in np.arange(target_label_index,0,-gap_)]  + [i  for i in np.arange(0,len(label) - target_label_index,gap_ )]
  return label_dist
#--

target_label_index = torch.where(label==1)[0][0].numpy()
[i for i in range(target_label_index,0,-1)] + [i for i in range(0,len(label) - target_label_index )]

[0, 1]

In [60]:
y = torch.tensor([label_to_dist_v1(la) for la in label])
print(y)

tensor([[5, 4, 3, 2, 1, 0, 1],
        [5, 4, 3, 2, 1, 0, 1]])


In [61]:
y/2

tensor([[2.5000, 2.0000, 1.5000, 1.0000, 0.5000, 0.0000, 0.5000],
        [2.5000, 2.0000, 1.5000, 1.0000, 0.5000, 0.0000, 0.5000]])

In [40]:
y = torch.tensor([label_to_dist(la) for la in label])
print(y)

tensor([[5, 4, 3, 2, 1, 0, 1],
        [5, 4, 3, 2, 1, 0, 1]])


In [15]:
y = torch.tensor([label_to_dist(la) for la in label])
print(y)
q = pdf_fn(y)
print(q)

tensor([[5, 4, 3, 2, 1, 0, 1],
        [5, 4, 3, 2, 1, 0, 1]])
tensor([[8.3879e-07, 7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01,
         1.3652e-01],
        [8.3879e-07, 7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01,
         1.3652e-01]])


In [16]:
# import matplotlib.pyplot as plt

# plt.bar([0,1,2,3,4,5,6],q.numpy())

In [17]:
prediction = torch.tensor( [0.1,0.1,
                            0.1,0.1,
                            0.1,0.4,
                            0.1] )

In [18]:
  # target 도 normalize 되어야 하나, => softmax 로 normalize 하면은 분포가 너무 고르게 된다.
# -->
target = q.clone()
target = target / sum(target)
#target = torch.tensor( [i/sum(target) for i in target] )
print(target)

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000]])


In [19]:
F.kl_div(prediction.log(), target, None, None, 'batchmean')

# 0.0046 <- prediction = ... 0.1, 0.4]
# -0.0129 <- prediction = ... 0.4, 0.1]

tensor(1.4114)

In [20]:
ce_target = torch.tensor([0,0,0,
                          0,0,1,
                          0],dtype=torch.float)
F.cross_entropy(prediction, ce_target)

tensor(1.6947)

# Need One hot embedding

In [21]:
F.one_hot(torch.arange(0, 5) % 3, num_classes=5)

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0]])

In [22]:
prediction

tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.4000, 0.1000])

In [23]:
label

tensor([[0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0]])

In [24]:
label_int = torch.tensor([1, 5, 4, 0, 2, 5, 2, 4])

In [25]:
F.one_hot(label_int,num_classes=7)

tensor([[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0]])

In [26]:
label_onehot = F.one_hot(label_int,num_classes=7)

In [27]:
label_onehot

tensor([[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0]])

In [28]:
[label_to_dist(i) for i in label_onehot]

[[1, 0, 1, 2, 3, 4, 5],
 [5, 4, 3, 2, 1, 0, 1],
 [4, 3, 2, 1, 0, 1, 2],
 [0, 1, 2, 3, 4, 5, 6],
 [2, 1, 0, 1, 2, 3, 4],
 [5, 4, 3, 2, 1, 0, 1],
 [2, 1, 0, 1, 2, 3, 4],
 [4, 3, 2, 1, 0, 1, 2]]

In [29]:
pdf_fn( torch.tensor( [label_to_dist(i) for i in label_onehot]) )

tensor([[1.3652e-01, 2.2508e-01, 1.3652e-01, 3.0461e-02, 2.5004e-03, 7.5506e-05,
         8.3879e-07],
        [8.3879e-07, 7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01,
         1.3652e-01],
        [7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01, 1.3652e-01,
         3.0461e-02],
        [2.2508e-01, 1.3652e-01, 3.0461e-02, 2.5004e-03, 7.5506e-05, 8.3879e-07,
         3.4279e-09],
        [3.0461e-02, 1.3652e-01, 2.2508e-01, 1.3652e-01, 3.0461e-02, 2.5004e-03,
         7.5506e-05],
        [8.3879e-07, 7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01,
         1.3652e-01],
        [3.0461e-02, 1.3652e-01, 2.2508e-01, 1.3652e-01, 3.0461e-02, 2.5004e-03,
         7.5506e-05],
        [7.5506e-05, 2.5004e-03, 3.0461e-02, 1.3652e-01, 2.2508e-01, 1.3652e-01,
         3.0461e-02]])

In [30]:
labels_ = pdf_fn( torch.tensor( [label_to_dist(i) for i in label_onehot]) )

In [31]:
torch.sum( labels_[0] )

tensor(0.5312)

In [32]:
labels_[0] / torch.sum( labels_[0])

tensor([2.5702e-01, 4.2376e-01, 2.5702e-01, 5.7349e-02, 4.7075e-03, 1.4215e-04,
        1.5792e-06])

In [33]:
torch.sum( labels_[0] / torch.sum( labels_[0]) )

tensor(1.)

In [34]:
#--- matrix approach 

In [35]:
labels_.shape

torch.Size([8, 7])

In [36]:
labels_.mean(dim=1, keepdim=True)

tensor([[0.0759],
        [0.0759],
        [0.0802],
        [0.0564],
        [0.0802],
        [0.0759],
        [0.0802],
        [0.0802]])